<a href="https://colab.research.google.com/github/cat0ros/robotics-control-3DOF/blob/master/fk_ik_python_3dof.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание по управлению роботами №1
Выполнил: Романов Ростислав, группа: АДБ-20-09

<img src= "https://drive.google.com/uc?export=view&id=1PpzfCIzka8X_ljyS4piV1AKeBfmu2AZ3" alt="inverse" width="400" />

***Рисунок 1: Пространственное представление схемы манипулятора***

<img src= "https://drive.google.com/uc?export=view&id=1zDL-aUGP3qxmC3ibMRZ7z9wZ6hDe7knu" alt="inverse" width="400" />

***Рисунок 2: Кинематическая схема манипулятора***

## Параметры и описание робота
Длины звеньев робота:

In [ ]:
links_length = [0.5, 0.5, 0.2]

## Библиотеки для работы

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
from numpy.linalg import inv
import math
from IPython.display import HTML
%matplotlib notebook

## Решение прямой задачи кинематики
**Составление DH-матриц**

Матрицы Денавита-Хартенберга:

In [ ]:
def get_dh_matrix(theta, alpha, r, d):
  alpha = (np.pi/180) * alpha
  theta = (np.pi/180) * theta

  ct = np.cos(theta)
  st = np.sin(theta)
  sa = np.sin(alpha)
  ca = np.cos(alpha)

  dh_matrix = np.array([
      [ct,  -st * ca, st * sa, r * ct],
      [st, ct *  ca, -ct * sa, r * st],
      [0, sa, ca, d],
      [0, 0, 0, 1]
  ])

  return dh_matrix

In [ ]:
def forward_kinematics(generilized):
  r = [0, 0, -links_length[2]]
  d = [links_length[0], links_length[1] + generilized[1], 0]
  theta = [generilized[0] - 90, -90, generilized[2] + 90]
  alpha = [-90, -90, 0]

  #links_length[2]

  matrix = np.eye(4)
  for i in range(len(theta)):
    matrix = matrix @ get_dh_matrix(theta[i], alpha[i], r[i], d[i])
    print(matrix)

  return matrix

In [ ]:
forward_kinematics([-30, 0.5, 90])

[[-5.00000000e-01  5.30287619e-17  8.66025404e-01  0.00000000e+00]
 [-8.66025404e-01 -3.06161700e-17 -5.00000000e-01  0.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00  6.12323400e-17  5.00000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[-8.36449319e-17 -8.66025404e-01 -5.00000000e-01  8.66025404e-01]
 [-2.24125920e-17  5.00000000e-01 -8.66025404e-01 -5.00000000e-01]
 [ 1.00000000e+00 -6.12323400e-17 -6.12323400e-17  5.00000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[-2.24125920e-17  8.66025404e-01 -5.00000000e-01  8.66025404e-01]
 [ 8.36449319e-17 -5.00000000e-01 -8.66025404e-01 -5.00000000e-01]
 [-1.00000000e+00 -6.12323400e-17 -6.12323400e-17  7.00000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


array([[-2.24125920e-17,  8.66025404e-01, -5.00000000e-01,
         8.66025404e-01],
       [ 8.36449319e-17, -5.00000000e-01, -8.66025404e-01,
        -5.00000000e-01],
       [-1.00000000e+00, -6.12323400e-17, -6.12323400e-17,
         7.00000000e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

# Решение обратной задачи кинематики

In [ ]:
def IK(x, y, z, l1, l2, l3):
  theta1 = np.arctan2(y, x)
  theta3_1 = np.arcsin((z - l1) / l3)
  theta3_2 = np.pi - np.arcsin((z - l1) / l3)
  theta3_sol = [theta3_1, theta3_2]

  sqrt_xy = np.float64((x**2 + y**2)**(1/2))
  l3_x0y0_1 = np.float64(l3) * np.cos(theta3_1)
  l3_x0y0_2 = np.float64(l3) * np.cos(theta3_2)

  d2_1 = sqrt_xy - l3_x0y0_1 - l2
  d2_2 = sqrt_xy - l3_x0y0_2 - l2
  d2_sol = [d2_1, d2_2]

  for i in range(len(theta3_sol)):
    q = [np.round(np.rad2deg(theta1), 2), np.round(d2_sol[i], 5), np.round(np.rad2deg(theta3_sol[i]), 2)]
    yield q

In [ ]:
fk_sol = forward_kinematics([60, 1, 10])
fk_sol

[[ 8.66025404e-01  3.06161700e-17  5.00000000e-01  0.00000000e+00]
 [-5.00000000e-01  5.30287619e-17  8.66025404e-01  0.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00  6.12323400e-17  5.00000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 2.24125920e-17 -5.00000000e-01  8.66025404e-01  7.50000000e-01]
 [-8.36449319e-17 -8.66025404e-01 -5.00000000e-01  1.29903811e+00]
 [ 1.00000000e+00 -6.12323400e-17 -6.12323400e-17  5.00000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[-4.92403877e-01  8.68240888e-02  8.66025404e-01  8.48480775e-01]
 [-8.52868532e-01  1.50383733e-01 -5.00000000e-01  1.46961181e+00]
 [-1.73648178e-01 -9.84807753e-01 -6.12323400e-17  5.34729636e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


array([[-4.92403877e-01,  8.68240888e-02,  8.66025404e-01,
         8.48480775e-01],
       [-8.52868532e-01,  1.50383733e-01, -5.00000000e-01,
         1.46961181e+00],
       [-1.73648178e-01, -9.84807753e-01, -6.12323400e-17,
         5.34729636e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [ ]:
tcp_pos = fk_sol @ np.array([[0], [0], [0], [1]])
tcp_pos

array([[0.84848078],
       [1.46961181],
       [0.53472964],
       [1.        ]])

In [ ]:
x_f, y_f, z_f = tcp_pos[0, 0], tcp_pos[1, 0], tcp_pos[2, 0]
l1, l2, l3 = links_length[0], links_length[1], links_length[2]

In [ ]:
x_f

0.8484807753012207

In [ ]:
y_f

1.4696118120671466

In [ ]:
z_f

0.5347296355333861

In [ ]:
for i in IK(x_f, y_f, z_f, l1, l2, l3):
  print(i)

[60.0, 1.0, 10.0]
[60.0, 1.39392, 170.0]
